<a href="https://colab.research.google.com/github/panizAZ/Data_science_journey/blob/main/PanizAzarnia_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paniz Azarnia _ HW1

<div dir="rtl" style="text-align: right;">

## شبیه‌سازی پرندگان (Boids)

الگوریتم **Boids** مدل ساده‌ای برای شبیه‌سازی رفتار جمعی پرندگان و ماهی‌هاست که توسط **Craig Reynolds** در سال ۱۹۸۶ معرفی شد. در این مدل:

1. **تجمع (Cohesion)**: هر بوید تمایل دارد به مرکز جرم بویدهای پیرامون خود نزدیک شود. با محاسبهٔ میانگین موقعیت همسایه‌ها و حرکت به سمت آن انجام می‌شود.

2. **هم‌راستایی (Alignment)**: هر بوید سرعت و جهت حرکت خود را با میانگین بردار سرعت همسایه‌ها هم‌راستا می‌کند.

3. **دوری (Separation)**: اگر بویدها خیلی به هم نزدیک شوند، نیروی دورکننده‌ای به آن‌ها اعمال می‌شود تا از برخورد جلوگیری کند. معمولاً از یک تابع معکوس فاصله یا تابعی مشابه استفاده می‌شود.

#### جریان اجرای هر گام زمانی:
- محاسبهٔ بردار شتاب بر اساس سه قانون بالا (هر یک با وزن مخصوص).
- به‌روزرسانی سرعت (`velocity`) با اعمال شتاب (محدود به `max_speed`).
- جابجایی موقعیت (`position`) بر اساس سرعت جدید.

**پارامترهای قابل تنظیم:**
- `neighbor_radius`: شعاع دید برای تعیین همسایه‌ها
- `max_speed`: بیشینه سرعت قابل قبول
- `max_force`: بیشینه نیروی (شتاب) قابل اعمال
- وزن قوانین: `weight_cohesion`, `weight_alignment`, `weight_separation`

**چالش پیاده‌سازی:**
1. با پایتون خالص (حلقه‌ها و ساختارهای لیستی) الگوریتم Boids را پیاده‌سازی کنید.
2. با **NumPy vectorization** (عبارات برداری بدون حلقهٔ صریح) همان شبیه‌سازی را بنویسید.
گام‌های زمانی و تعداد و شرایط اولیه پرنده‌ها باید دلخواه باشد
</div>



In [ ]:
# Code one without numpy
import math
import random
class Boid:
    def __init__(self, x=None, y=None, vx=None, vy=None):
        self.x = x if x is not None else random.uniform(0, 500)  # Random x position between 0 and 500
        self.y = y if y is not None else random.uniform(0, 500)  # Random y position between 0 and 500
        self.vx = vx if vx is not None else random.uniform(-5, 5)  # Random velocity in the range [-5, 5]
        self.vy = vy if vy is not None else random.uniform(-5, 5) # Random velocity in the range [-5, 5]


boids = [Boid() for _ in range(100)]
neighbor_radius = 40
private_radius = 8
weight_separation = 0.05
weight_cohesion =  0.0005
weight_alignment = 0.05
max_speed = 6
max_force = 1.5

for iteration in range(100):
    for boid in boids:

        close_dx = 0
        close_dy = 0
        xpos_avg = 0
        ypos_avg = 0
        xvel_avg = 0
        yvel_avg = 0
        neighbor_num = 0

        for others in boids:
            if others != boid:  # Skip self interaction
                #Seperation
                dx = others.x - boid.x
                dy = others.y - boid.y

                squared_distance = dx * dx + dy * dy
                # If other boids in the private range
                if squared_distance < private_radius ** 2:

                    # If so, calculate difference in x/y-coordinates to nearfield boid
                    close_dx += boid.x - others.x
                    close_dy += boid.y - others.y

                elif squared_distance < neighbor_radius ** 2:
                    # Cohesion
                    xpos_avg += others.x
                    ypos_avg += others.y
                    xvel_avg += others.vx
                    yvel_avg += others.vy

                    neighbor_num += 1

        if neighbor_num > 0:
            # Divide accumulator variables by number of boids in visual range
            xpos_avg /= neighbor_num
            ypos_avg /= neighbor_num
            xvel_avg /= neighbor_num
            yvel_avg /= neighbor_num

            ax = ((xpos_avg - boid.x) * weight_cohesion +
                 (xvel_avg - boid.vx) * weight_alignment +
                 close_dx * weight_separation)

            ay = ((ypos_avg - boid.y) * weight_cohesion +
                 (yvel_avg - boid.vy) * weight_alignment +
                 close_dy * weight_separation)

            acc_magnitude = math.sqrt(ax ** 2 + ay ** 2)

            # Enforce max force
            if acc_magnitude > max_force:
                 ax = (ax / acc_magnitude) * max_force
                 ay = (ay / acc_magnitude) * max_force

            boid.vx += ax
            boid.vy += ay
            speed = math.sqrt(boid.vx ** 2 + boid.vy ** 2)

            # Enforce max speeds
            if speed > max_speed:
                 boid.vx = (boid.vx/speed)*max_speed
                 boid.vy = (boid.vy/speed)*max_speed


        # Update boid's position
        boid.x = boid.x + boid.vx
        boid.y = boid.y + boid.vy


In [ ]:
# Code two with numpy
import numpy as np

# Number of boids
N = 100

neighbor_radius = 40
private_radius = 8
weight_separation = 0.05
weight_cohesion =  0.0005
weight_alignment = 0.05
max_speed = 6
max_force = 1.5

# Initialize
positions = np.random.rand(N, 2) * 500  # [0, 500]
velocities = (np.random.rand(N, 2) - 0.5) * 10  # [-5, 5]


for iteration in range(100):
    # Compute differences between positions (broadcasting)
    diffs = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]  # (N, N, 2)
    dists_squared = np.sum(diffs ** 2, axis=2)  # (N, N)

    # Find neighbors and close boids
    neighbors = (dists_squared > 0) & (dists_squared < neighbor_radius ** 2)  # (N, N)
    close = dists_squared < private_radius ** 2  # (N, N)

    # Separation force (sum of vectors pointing away from neighbors)
    separation_vec = np.sum(diffs * close[:, :, np.newaxis], axis=1)  # (N, 2)

    # Cohesion: Move towards the average position of neighbors
    avg_positions = np.sum(positions[:, np.newaxis, :] * neighbors[:, :, np.newaxis], axis=1) / np.sum(neighbors, axis=1)[:, np.newaxis]
    cohesion_vec = avg_positions - positions

    # Alignment: Match the average velocity of neighbors
    avg_velocities = np.sum(velocities[:, np.newaxis, :] * neighbors[:, :, np.newaxis], axis=1) / np.sum(neighbors, axis=1)[:, np.newaxis]
    alignment_vec = avg_velocities - velocities

    acceleration = (weight_separation * separation_vec +
                    weight_cohesion * cohesion_vec +
                    weight_alignment * alignment_vec)

    # Limit acceleration (max_force)
    acc_mag = np.linalg.norm(acceleration, axis=1)
    exceed_mask = acc_mag > max_force
    acceleration[exceed_mask] = (acceleration[exceed_mask] /
                                 acc_mag[exceed_mask][:, np.newaxis]) * max_force

    new_velocities = velocities + acceleration

    # Limit speed (max_speed)
    speed = np.linalg.norm(new_velocities, axis=1)
    speed_mask = speed > max_speed
    new_velocities[speed_mask] = (new_velocities[speed_mask] /
                                  speed[speed_mask][:, np.newaxis]) * max_speed

    # Update positions
    positions += new_velocities


<ipython-input-4-ffd4d3240729>:33: RuntimeWarning: invalid value encountered in divide
  avg_positions = np.sum(positions[:, np.newaxis, :] * neighbors[:, :, np.newaxis], axis=1) / np.sum(neighbors, axis=1)[:, np.newaxis]
<ipython-input-4-ffd4d3240729>:37: RuntimeWarning: invalid value encountered in divide
  avg_velocities = np.sum(velocities[:, np.newaxis, :] * neighbors[:, :, np.newaxis], axis=1) / np.sum(neighbors, axis=1)[:, np.newaxis]


<div dir="rtl" style="text-align: right;">

## بازی زندگی کانوی (Game of Life)

بازی زندگی یک خودکار سلولی است روی یک شبکهٔ دوبعدی مربعی که توسط **John Conway** در سال ۱۹۷۰ معرفی شد. هر سلول دو وضعیت دارد:
- زنده (`1`)
- مرده (`0`)

**قوانین به‌روزرسانی وضعیت سلول در هر گام:**
1. **تنهایی**: سلول زنده با کمتر از دو همسایهٔ زنده، می‌میرد.
2. **تداوم حیات**: سلول زنده با دو یا سه همسایهٔ زنده زنده می‌ماند.
3. **ازدحام**: سلول زنده با بیش از سه همسایهٔ زنده می‌میرد.
4. **زایش**: سلول مرده با دقیقاً سه همسایهٔ زنده، زنده می‌شود.

هر سلول ۸ همسایه—شامل قطری‌ها—دارد. برای مرزها می‌توانید:
- **بسته (fixed)**: خانه‌های خارج شبکه مرده فرض شوند.
- **هم‌بسته (toroidal wrap)**: شبکه چرخشی باشد (لبه‌ها به هم متصل).

**الگوهای معروف:**
- **گلیدر (Glider)**: ساختار متحرک و خودپایداری که در شبکه حرکت می‌کند.
- **نوسانگر (Oscillator)**: الگوهای تکرارشونده دوره‌ای.
- **اسلایس ایستا (Still lifes)**: الگوهای پایدار و بدون تغییر.

**چالش پیاده‌سازی:**
1. با پایتون خالص و استفاده از حلقه‌های دوگانه روی ماتریس شبیه‌سازی کنید.
2. با **NumPy vectorization** و استفاده از توابع برداری (مثلاً `convolve` یا شمارش همسایه‌ها با عملگرهای آرایه‌ای) بدون حلقه‌های صریح پیاده‌سازی را انجام دهید.

تعداد خانه‌های شبکه مربعی و تعداد مرحله‌های زمانی باید دلخواه باشد.
</div>



In [ ]:
# Code one without numpy
import time
import os

class Cell:
    def __init__(self, x, y, value):
        self.x = x
        self.y = y
        self.value = value

N = 10
steps = 10  # Only plays 10 rounds

# Initializing values
cells = [[Cell(i, j, 0) for j in range(N)] for i in range(N)]
cells[1][2].value = 1
cells[2][3].value = 1
cells[3][1].value = 1
cells[3][2].value = 1
cells[3][3].value = 1

def print_cells(cells):
    for row in cells:
        print("".join(['⬛' if cell.value else '⬜' for cell in row]))
    print()

def step_game(cells):
    new_values = [[0]*N for _ in range(N)]

    for i in range(N):
        for j in range(N):
            num_neighbors = 0

            for neighbor_x in range(i - 1, i + 2):
                for neighbor_y in range(j - 1, j + 2):
                    if 0 <= neighbor_x < N and  0 <= neighbor_y < N:
                        if neighbor_x == i and neighbor_y == j:
                            continue
                        if cells[neighbor_x][neighbor_y].value == 1:
                            num_neighbors += 1

            # Implementing Rules
            if cells[i][j].value == 1:
                if num_neighbors < 2 or num_neighbors > 3:
                    new_values[i][j] = 0  # Dies
                else:
                    new_values[i][j] = 1  # Stays alive
            else:  # if already dead
                if num_neighbors == 3:
                    new_values[i][j] = 1  # Becomes alive

    # Upgrading values
    for i in range(N):
        for j in range(N):
            cells[i][j].value = new_values[i][j]

# Start
for step in range(steps):
    os.system("cls" if os.name == "nt" else "clear")
    print(f"Step {step + 1}")
    print_cells(cells)
    step_game(cells)
    time.sleep(0.5)


Step 1
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬛⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 2
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 3
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 4
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 5
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 6
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 7
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 8
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬛⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 9
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜

In [ ]:
# Code two with numpy
import time
import os
import numpy as np
from scipy.ndimage import convolve

def print_cells(cells):
    for row in cells:
        print("".join(['⬛' if cell else '⬜' for cell in row]))
    print()

def step_game(cells):
    # To calculate number of neighbors
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])

    # number of alive neighbors
    neighbors = convolve(cells, kernel, mode='constant', cval=0)

    # The rules
    new_values = (cells & (neighbors == 2)) | (neighbors == 3)

    return new_values.astype(int)


N = 10
steps = 10

# Initializing values
cells = np.zeros((N, N), dtype=int)
cells[1, 2] = 1
cells[2, 3] = 1
cells[3, 1] = 1
cells[3, 2] = 1
cells[3, 3] = 1

# Start
for step in range(steps):
    os.system("cls" if os.name == "nt" else "clear")
    print(f"Step {step + 1}")
    print_cells(cells)
    cells = step_game(cells)
    time.sleep(0.5)


Step 1
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬛⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 2
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 3
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 4
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 5
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 6
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 7
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 8
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬛⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜

Step 9
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜